In [ ]:
%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re
from pathlib import Path

In [ ]:
# time or frequency
TimeOrFrequency = 'Time'
#TimeOrFrequency = 'Frequency'

In [ ]:
KeyNotInclude = ['PhraseTextBlock', 'ScratchPad', 'Sleep', 'NextSuggestions', 'NextPhrase', 'Quit', 'YesQuestionResult', 'LeftShift']
TimeDwellOrig = 800

In [ ]:
def ComputeCharTyped(scratchPad, userKeys):
    phraseUser = list()
    phraseUserEnd = list()
    timeUser = list()
    
    # find phrases typed
    for row_ind in range(0, len(scratchPad)):
        if row_ind == 0:
            timeUser.append(scratchPad[row_ind][0])
        if row_ind!= 0 and scratchPad[row_ind][1] == '':
            phraseUser.append(scratchPad[row_ind-1])
            timeUser.append(scratchPad[row_ind][0])
                        
    for row_ind2 in range(0, len(phraseUser)):
        if len(phraseUser[row_ind2])>2:
            phraseUser[row_ind2][1] = ''.join(phraseUser[row_ind2][1:])
        phraseUserEnd.append(phraseUser[row_ind2][1])
    
    # start time of typing:
    LetterLookedAt = [userKeys[letterInd] for letterInd in range(0,len(userKeys)) if userKeys[letterInd][1] not in KeyNotInclude and 'Suggestion' not in userKeys[letterInd][1]]
    
    #print(LetterLookedAt[0])
    
    headStartTime,sep,tail = LetterLookedAt[0][0].partition('+')
    headEndTime,sep,tail = timeUser[-1].partition('+')
    timeOfTyping = datetime.datetime.strptime(re.sub('[:.T]','-',headEndTime[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]','-',headStartTime[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    # print(timeOfTyping)
    #print(phraseUserEnd)
    return timeOfTyping if TimeOrFrequency == 'Time' else len(''.join(phraseUserEnd)) 

In [ ]:
def ComputeCharNotActivated(userKeys, TimeOrFrequency, timeDwellOrig):
    
    timeDwell = timeDwellOrig
    
    # list of non-special character keys
    LetterLookedAt = [userKeys[letterInd] for letterInd in range(0,len(userKeys)) if userKeys[letterInd][1] not in KeyNotInclude and 'Suggestion' not in userKeys[letterInd][1]]
    LetterLookingAt = list()
           
    # create list of unique keys looked at
    for keyInd in range(0, len(LetterLookedAt)):
        if keyInd == 0:
            theLetter = LetterLookedAt[keyInd][1]
            LetterLookingAt.append(LetterLookedAt[keyInd])
            continue
                        
        checkLetter = LetterLookedAt[keyInd][1]
                    
        if theLetter != checkLetter:
            theLetter = checkLetter
            LetterLookedAtList.append(LetterLookingAt)
            LetterLookingAt = list()
            LetterLookingAt.append(LetterLookedAt[keyInd])
        
        else:
            LetterLookingAt.append(LetterLookedAt[keyInd])
    
    
    # combine lists within LetterLookedAtList if they were the continued look - can happen even with 2 different key looks 
    # in between
    firstKeyOnList = 0 
    nKey = -1
    mergeKeysIndex = list()
    for keyList in LetterLookedAtList:
        nKey = nKey + 1
        # if first key looked at, continue
        if firstKeyOnList == 0:
            firstKeyOnList = 1
            continue
        # cant continue with less than 3 keys looked at
        if nKey < 3:
            continue
        
        # check if the current key and the second to last one was the same
        if keyList[0][1] == LetterLookedAtList[nKey-2][0][1]:
            # make sure the second last key did not have 0 progress
            if float(LetterLookedAtList[nKey-2][-1][2]) != 0:
            
                # now check against second to last key looked at, and if it is continued
                if float(keyList[0][2]) > float(LetterLookedAtList[nKey-2][-1][2]):
                    # record the index of the two merging lists
                    mergeKeysIndex.append([nKey-2, nKey])
                    

    
    #print(len(LetterLookedAtList))
    #print(len(mergeKeysIndex))
    # merge the lists
    nKey = len(mergeKeysIndex)
    while nKey > 0:
        nKey = nKey - 1
        mergeList = mergeKeysIndex[nKey]
        LetterLookedAtList[mergeList[0]] = LetterLookedAtList[mergeList[0]] + LetterLookedAtList[mergeList[1]]
        LetterLookedAtList.remove(LetterLookedAtList[mergeList[1]])
    
    #print(len(LetterLookedAtList))
    
    
    
    # find if dwell time was changed or not, and change the consecutive progress into time with current dwell time
    nKey = -1
    
    # compute dwell time, if changes
    if TimeOrFrequency == 'Time':
        for key in LetterLookedAtList:
            nKey = nKey + 1
            if key[0][1] == 'IncreaseDwellTime':
                # create a list of the progress, to check if it was actually selected
                progressList = [float(row[2]) for row in key]
                if 1 in progressList:
                    timeDwell = timeDwell + 100

            elif key[0][1] == 'DecreaseDwellTime':
                # create a list of the progress, to check if it was actually selected
                progressList = [float(row[2]) for row in key]
                if 1 in progressList:
                    timeDwell = timeDwell - 100
                    
            else:
                # change the progress to time, depending on the time dwell
                for nRow in range(0, len(key)):
                    LetterLookedAtList[nKey][nRow].append(str(float(LetterLookedAtList[nKey][nRow][2])*timeDwell))
    
    print(len(LetterLookedAtList))
    
    # now remove all the keys that were successfully pressed (most times it will also include dwell time change keys)
    nKey = len(LetterLookedAtList)
    # if progress was 1, the key is removed
    while nKey > -1:
        nKey = nKey - 1
        key = LetterLookedAtList[nKey]
        #print(nKey)
        #print(key)
        for progressInd in key:
            if float(progressInd[2]) == 1:
                LetterLookedAtList.remove(LetterLookedAtList[nKey])
                break
    
    
    #print(LetterLookedAtList)
        
    # Make sure that lists with Dwell time changes are removed
    nKey = len(LetterLookedAtList)
    while nKey > -1:
        nKey = nKey - 1
        key = LetterLookedAtList[nKey]
        if key[0][1] == 'IncreaseDwellTime':
            LetterLookedAtList.remove(LetterLookedAtList[nKey])
        elif key[0][1] == 'DecreaseDwellTime':
            LetterLookedAtList.remove(LetterLookedAtList[nKey])            
    
    # Calculate the time
    if TimeOrFrequency == 'Time':
        timeAttending = 0
        for key in LetterLookedAtList:
            nonZeroProgress = 0
            nKey = len(key)
            while nKey > -1:
                nKey = nKey - 1
                if float(key[nKey][2]) > 0:
                    timeAttending = timeAttending + float(key[nKey][3])
                    break
                    
    #print(timeAttending)
    
    #print(LetterLookedAtList)
    
    return timeAttending if TimeOrFrequency == 'Time' else len(LetterLookedAtList)

In [ ]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName[0], DataEvaluated[ind[0]]])
    

In [ ]:
if TimeOrFrequency == 'Time':
    metricComputed = 'attended_but_not_selected_rate_time'
else:
    metricComputed = 'attended_but_not_selected_rate_frequency'
AttendedButNotSelectedRateList = list()
Names = list()

subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    userKeys = None
    scratchPad = None
    #print(root)
    if not dirs:
        
        if 'notCompleted' in root:
            continue
        if 'tb' in root or 'joha' in root:
            continue

        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            else:
                continue
            
            if userKeys and scratchPad:
                
                # save only the name of the subject and the date part of the file
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                print(a.findall(root))
                Names.append(a.findall(root)[0])
                
                #print(scratchPad)
                CharNotActivated = ComputeCharNotActivated(userKeys, TimeOrFrequency, TimeDwellOrig)
                
                # number of characters typed or time spent typing
                CharActivated = ComputeCharTyped(scratchPad, userKeys)
                
                if TimeOrFrequency == 'Time':
                    AttendedButNotSelectedMeasure = (datetime.timedelta(seconds = CharNotActivated/1000))/CharActivated
                    print(CharNotActivated/1000, 's', '  ', CharActivated, '  ', AttendedButNotSelectedMeasure)
                else:
                    AttendedButNotSelectedMeasure = float(CharNotActivated/CharActivated)
                    print(CharNotActivated, '  ', CharActivated, '  ', AttendedButNotSelectedMeasure)

                    
                
                AttendedButNotSelectedRateList.append(AttendedButNotSelectedMeasure)
                
            
if AttendedButNotSelectedRateList:
    AddDataToFile(metricComputed, Names, AttendedButNotSelectedRateList)